## Task 1

In [1]:
import os
import requests
from openai import OpenAI
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
!pip show tiktoken

Name: tiktoken
Version: 0.9.0
Summary: tiktoken is a fast BPE tokeniser for use with OpenAI's models
Home-page: https://github.com/openai/tiktoken
Author: Shantanu Jain
Author-email: shantanu@openai.com
License: MIT License

Copyright (c) 2022 OpenAI, Shantanu Jain

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULA

In [17]:
!pip uninstall elasticsearch --yes

Found existing installation: elasticsearch 9.0.2
Uninstalling elasticsearch-9.0.2:
  Successfully uninstalled elasticsearch-9.0.2


In [12]:
!pip show elasticsearch

Name: elasticsearch
Version: 9.0.2
Summary: Python client for Elasticsearch
Home-page: https://github.com/elastic/elasticsearch-py
Author: 
Author-email: Elastic Client Library Maintainers <client-libs@elastic.co>
License-Expression: Apache-2.0
Location: /home/codespace/miniconda3/lib/python3.13/site-packages
Requires: elastic-transport, python-dateutil, typing-extensions
Required-by: 


In [19]:
!pip install elasticsearch==9.0.2

  Using cached elasticsearch-9.0.2-py3-none-any.whl.metadata (8.4 kB)
Using cached elasticsearch-9.0.2-py3-none-any.whl (914 kB)


In [14]:
client = OpenAI(base_url="https://api.proxyapi.ru/openai/v1",
                api_key=os.environ['KEY'])

In [ ]:
resp = client.chat.completions.create(
    model='gpt-4o-mini',
    max_completion_tokens=5,
    messages=[{'role': 'user', 'content': 'как приготовить лосося?'}]
)
resp.choices[0].messege.content

In [5]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [6]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [7]:
es_client = Elasticsearch('http://localhost:9200'
                         )

In [8]:
es_client.info()

ObjectApiResponse({'name': '175042a80b3b', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'rfGryj34Siur4yiauXF7Lg', 'version': {'number': '9.0.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '0a58bc1dc7a4ae5412db66624aab968370bd44ce', 'build_date': '2025-05-28T10:06:37.834829258Z', 'build_snapshot': False, 'lucene_version': '10.1.0', 'minimum_wire_compatibility_version': '8.18.0', 'minimum_index_compatibility_version': '8.0.0'}, 'tagline': 'You Know, for Search'})

In [9]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}
index_name = 'course-questions'

In [13]:
es_client.info()

ObjectApiResponse({'name': '175042a80b3b', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'rfGryj34Siur4yiauXF7Lg', 'version': {'number': '9.0.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '0a58bc1dc7a4ae5412db66624aab968370bd44ce', 'build_date': '2025-05-28T10:06:37.834829258Z', 'build_snapshot': False, 'lucene_version': '10.1.0', 'minimum_wire_compatibility_version': '8.18.0', 'minimum_index_compatibility_version': '8.0.0'}, 'tagline': 'You Know, for Search'})

In [ ]:
es_client.indices.create(index=index_name, body=index_settings)

In [ ]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

In [14]:
query = 'How do execute a command on a Kubernetes pod?'

In [15]:
search_query = {
    "size": 1,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text",],
                    "type": "best_fields"
                }
            },
        }
    }
}

In [16]:
response = es_client.search(index=index_name, body=search_query)

In [17]:
response['hits']['hits']

[{'_index': 'course-questions',
  '_id': 'WhA8b5cBrtm5cMNyYeP7',
  '_score': 44.50556,
  '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I debug a docker container?',
   'course': 'machine-learning-zoomcamp'}}]

In [18]:
query = 'How do copy a file to a Docker container?'

In [19]:
def search_elasic(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
    
    response = es_client.search(index=index_name, body=search_query)
    
    results = [hit['_source'] for hit in response['hits']['hits']]
    
    return results

In [20]:
results = search_elasic(query)
results

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

In [21]:
def get_promt(query, results):

    context = ''
    for doc in results:
        context += context + f"Q: {doc['question']}\nA: {doc['text']}\n\n"
    
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    
    QUESTION: {question}
    
    CONTEXT:
    {context}
    """.strip()
    
    promt = prompt_template.format(question=query, context=context)

    return promt

In [22]:
get_promt(query, results)

'You\'re a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.\n    Use only the facts from the CONTEXT when answering the QUESTION.\n\n    QUESTION: How do copy a file to a Docker container?\n\n    CONTEXT:\n    Q: How do I debug a docker container?\nA: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)\n\nQ: How do I debug a docker container?\nA: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)\n\nQ: How do I 

In [23]:
len(get_promt(query, results))

2981

In [24]:
def use_llm(query, promt):
    client = OpenAI(base_url="https://api.proxyapi.ru/openai/v1",
                    api_key=os.environ['KEY'])
    resp = client.chat.completions.create(
        model='gpt-4o-mini',
        max_completion_tokens=5,
        messages=[{'role': 'user', 'content': promt}]
    )
    return resp.choices[0].messege.content

In [27]:
def ask_llm(query):
    results = search_elasic(query)
    promt = get_promt(query, results)
    answer = use_llm(query, promt)
    return answer

In [34]:
ask_llm('How do copy a file to a Docker container?')

KeyError: 'KEY'

In [32]:
!pip install tiktoken

In [33]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o")

ModuleNotFoundError: No module named 'tiktoken'

## Task 2

In [5]:
!pip install fastembed

  Using cached fastembed-0.7.1-py3-none-any.whl.metadata (10 kB)
  Using cached huggingface_hub-0.33.1-py3-none-any.whl.metadata (14 kB)
  Using cached loguru-0.7.3-py3-none-any.whl.metadata (22 kB)
  Using cached tokenizers-0.21.2-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
  Using cached hf_xet-1.1.5-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (879 bytes)
  Using cached coloredlogs-15.0.1-py2.py3-none-any.whl.metadata (12 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached protobuf-6.31.1-cp39-abi3-manylinux2014_x86_64.whl.metadata (593 bytes)
  Using cached humanfriendly-10.0-py2.py3-none-any.whl.metadata (9.2 kB)
Using cached fastembed-0.7.1-py3-none-any.whl (100 kB)
Using cached huggingface_hub-0.33.1-py3-none-any.whl (515 kB)
Using cached loguru-0.7.3-py3-none-any.whl (61 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 49.9 MB/s eta 0:00:00:00:01
Using cached tokenizers-0

In [4]:
import fastembed

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from fastembed import TextEmbedding

In [6]:
import sys
print(sys.executable)

/home/codespace/.python/current/bin/python


In [2]:
!pip list | grep fastembed

In [9]:
!pip install -q "qdrant-client[fastembed]>=1.14.2"


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [7]:
from qdrant_client import QdrantClient, models

In [32]:
client = QdrantClient("http://localhost:6333")

In [9]:
import requests

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

In [10]:
import json

EMBEDDING_DIMENSIONALITY = 512

for model in TextEmbedding.list_supported_models():
    if model["dim"] == EMBEDDING_DIMENSIONALITY:
        print(json.dumps(model, indent=2))

{
  "model": "BAAI/bge-small-zh-v1.5",
  "sources": {
    "hf": "Qdrant/bge-small-zh-v1.5",
    "url": "https://storage.googleapis.com/qdrant-fastembed/fast-bge-small-zh-v1.5.tar.gz",
    "_deprecated_tar_struct": true
  },
  "model_file": "model_optimized.onnx",
  "description": "Text embeddings, Unimodal (text), Chinese, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year.",
  "license": "mit",
  "size_in_GB": 0.09,
  "additional_files": [],
  "dim": 512,
  "tasks": {}
}
{
  "model": "Qdrant/clip-ViT-B-32-text",
  "sources": {
    "hf": "Qdrant/clip-ViT-B-32-text",
    "url": null,
    "_deprecated_tar_struct": false
  },
  "model_file": "model.onnx",
  "description": "Text embeddings, Multimodal (text&image), English, 77 input tokens truncation, Prefixes for queries/documents: not necessary, 2021 year",
  "license": "mit",
  "size_in_GB": 0.25,
  "additional_files": [],
  "dim": 512,
  "tasks": {}
}
{
  "model": "jinaai/jina-embeddings-v2-small-e

In [11]:
model_handle = "jinaai/jina-embeddings-v2-small-en"

In [12]:
# Define the collection name
collection_name = "homework"

# Create the collection with specified vector parameters
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)

ResponseHandlingException: [Errno 111] Connection refused

In [13]:
text = 'I just discovered the course. Can I join now?'

x = models.PointStruct(
    id=0,
    vector=models.Document(text=text, model=model_handle), #embed text locally with "jinaai/jina-embeddings-v2-small-en" from FastEmbed
    payload={
        "text": text,
    } #save all needed metadata fields
)
x.vector

Document(text='I just discovered the course. Can I join now?', model='jinaai/jina-embeddings-v2-small-en', options=None)

In [14]:
model_emb = TextEmbedding(model_name=model_handle)
embeddings = list(model_emb.embed(text))

Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00,  5.11it/s]


In [15]:
min(embeddings[0])

np.float64(-0.11726373885183883)

In [20]:
import numpy as np
np.linalg.norm(embeddings[0])

np.float64(1.0)

In [21]:
embeddings[0].dot(embeddings[0])

np.float64(1.0000000000000002)

In [22]:
doc = 'Can I still join the course after the start date?'
q = list(model_emb.embed(doc))[0]

In [23]:
embeddings[0].dot(q) / np.linalg.norm(embeddings[0]) / np.linalg.norm(q)

np.float64(0.9008528895674548)

In [24]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
 'section': 'General course-related questions',
 'question': 'Course - Can I still join the course after the start date?',
 'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
 'section': 'General course-related questions',
 'question': 'Course - Can I follow the course after it finishes?',
 'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
 'section': 'General course-related questions',
 'question': 'Course - What can I do before the course starts?',
 'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
 'section': 'General course-related questions',
 'question': 'How can we contribute to the course?',
 'course': 'data-engineering-zoomcamp'}]

In [25]:
for i, doc in enumerate(documents):
    q = list(model_emb.embed(doc['text']))[0]
    print(i, embeddings[0].dot(q) / np.linalg.norm(embeddings[0]) / np.linalg.norm(q))
    print()


0 0.7629684696540238

1 0.8182378150042889

2 0.8085397398734189

3 0.7133079015686243

4 0.7304499234333598



In [26]:
for i, doc in enumerate(documents):
    new_data = doc['question'] + ' ' + doc['text']
    q = list(model_emb.embed(new_data))[0]
    print(i, embeddings[0].dot(q) / np.linalg.norm(embeddings[0]) / np.linalg.norm(q))
    print()


0 0.8514543236908069

1 0.8436594159113068

2 0.840828704850256

3 0.7755157969663908

4 0.8086007795043938



In [28]:
import json

DIMs = []

for model in TextEmbedding.list_supported_models():
    if model["dim"] not in DIMs:
        DIMs.append(model["dim"])
DIMs

[768, 1024, 384, 512]

In [29]:
model = 'BAAI/bge-small-en'

import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [34]:
# Define the collection name
collection_name = "machine-learning-zoomcamp"

# Create the collection with specified vector parameters
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=384,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)

True

In [35]:
points = []
id = 0

for course in documents_raw:
    for doc in course['documents']:
        text = doc['question'] + ' ' + doc['text']
        point = models.PointStruct(
            id=id,
            vector=models.Document(text=text, model=model), #embed text locally with "jinaai/jina-embeddings-v2-small-en" from FastEmbed
            payload={
                "text": text,
                "section": doc['section'],
                "course": course['course']
            } #save all needed metadata fields
        )
        points.append(point)

        id += 1

In [36]:
client.upsert(
    collection_name=collection_name,
    points=points
)

Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00,  8.59it/s]


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [37]:
def search(query, limit=1):

    results = client.query_points(
        collection_name=collection_name,
        query=models.Document( #embed the query text locally with "jinaai/jina-embeddings-v2-small-en"
            text=query,
            model=model
        ),
        limit=limit, # top closest matches
        with_payload=True #to get metadata in the results
    )

    return results

In [38]:
question = 'I just discovered the course. Can I join now?'

In [39]:
result = search(question)
result

QueryResponse(points=[ScoredPoint(id=449, version=0, score=0.8703172, payload={'text': 'The course has already started. Can I still join it? Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.', 'section': 'General course-related questions', 'course': 'machine-learning-zoomcamp'}, vector=None, shard_key=None, order_value=None)])